<a href="https://colab.research.google.com/github/AdhamSalih/DeepBallTAU/blob/master/ModelsEvaluation(excluding_MF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Project - Training Deep Network for ball detection with pytorch**
This Project includes an evaluation of four models for Ball Detection
1. DeepBall 
2. DeepBall with classification layer
3. DeepBall - with relaxed loss
4. DeepBall with classification layer - with relaxed loss

# General setup

Mounting the drive

In [0]:
from google.colab import drive, files
drive.mount("/content/drive")
folderpath = '/content/drive/My Drive/Project_300746930_034915504/'


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Installing all the needed lib.

In [0]:
# !pip install torch numpy torchvision matplotlib tabulate
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms, utils, datasets
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import numpy as np
from tabulate import tabulate
import pickle
import cv2


#Dataset Loading

In [0]:
idx = 0
train_batch = pickle.load( open( folderpath + "Dataset/test/test_batch_file" + str(idx)+".data", "rb" ) )
Images = train_batch[0]
Labels = train_batch[1][:,0]
while idx < 2:
  idx += 1
  train_batch = pickle.load( open( folderpath + "Dataset/test/test_batch_file"+str(idx)+".data", "rb" ) )
  Images = torch.cat((Images,train_batch[0]))
  Labels = torch.cat((Labels,train_batch[1][:,0]))
  print(idx)
  del train_batch
test_dataset = TensorDataset(Images,Labels)
del Images,Labels

1
2


**Hyperparameters**

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device != "cpu":
  print("GPU is available")
else:
  print("Work on CPU")
BN = True
DP = 0.2

GPU is available


# Network Classes

DeepBall

In [0]:
class DeepBall(nn.Module):
    def __init__(self):
        super(DeepBall, self).__init__()
        # the 1st conv. block 
        self.conv11 = nn.Conv2d(3, 8, 3,stride=2,padding=1)
        self.conv12 = nn.Conv2d(8, 8, 3,stride=1,padding=1) 
        
         # pooling size 2
        self.pool = nn.MaxPool2d(2, 2) 

        # the 2nd conv. block 
        self.conv21 = nn.Conv2d(8, 16, 3,padding=1)
        self.conv22 = nn.Conv2d(16, 16, 3,padding=1)
        
        # the 3rd conv. block 
        self.conv31 = nn.Conv2d(16, 32, 3,padding=1)
        self.conv32 = nn.Conv2d(32, 32, 3,padding=1)
        
        # the 4th conv. block 
        self.conv41 = nn.Conv2d(56, 56, 3,padding=1)
        self.conv42 = nn.Conv2d(56, 2, 3,padding=1)
        
        # Upsampling
        self.ups24 = nn.Upsample(scale_factor=2)
        self.ups34 = nn.Upsample(scale_factor=4)
        
        self.bn11 = nn.BatchNorm2d(8)
        self.bn12 = nn.BatchNorm2d(8)
        self.bn21 = nn.BatchNorm2d(16)
        self.bn22 = nn.BatchNorm2d(16)
        self.bn31 = nn.BatchNorm2d(32)
        self.bn32 = nn.BatchNorm2d(32)
        self.bn41 = nn.BatchNorm2d(56)
        self.bn42 = nn.BatchNorm2d(2)
        
    def forward(self, x):
      # 1st block
      if x.size()[2]!=272 or x.size()[3]!=480:
        x = F.interpolate(x, size=(272,480))
      
      x1 = F.relu(self.bn11(self.conv11(x)))
      x1 = F.relu(self.bn12(self.conv12(x1)))
      x1 = self.pool(x1)
      
      # 2nd Block
      x2 = F.relu(self.bn21(self.conv21(x1)))
      x2 = F.relu(self.bn22(self.conv22(x2)))
      x2 = self.pool(x2)
      
      # 3rd Block
      x3 = F.relu(self.bn31(self.conv31(x2)))
      x3 = F.relu(self.bn32(self.conv32(x3)))
      x3 = self.pool(x3)

      # 4th Block
      x2 = self.ups24(x2)
      x3 = self.ups34(x3)
      x4 = torch.cat([x2,x3,x1],dim=1)
      x4 = F.relu(self.bn41(self.conv41(x4)))
      x4 = F.relu(self.bn42(self.conv42(x4)))  
      
      out = F.log_softmax(x4,dim=1)
      if out.size()[2]!=68 or out.size()[3]!=120:
        out = F.interpolate(out, size=(68,120))
      del x1,x2,x3,x4
      torch.cuda.empty_cache()
      return out

C-DeepBall

In [0]:
class DeepBallClass(nn.Module):
    def __init__(self, BN = True,DP = 0):
        super(DeepBallClass, self).__init__()
        self.BN = BN
        self.DP = DP
        # the 1st conv. block 
        self.conv11 = nn.Conv2d(3, 8, 3,stride=2,padding=1)
        
        self.conv12 = nn.Conv2d(8, 8, 3,stride=1,padding=1) 
        
         # pooling size 2
        self.pool = nn.MaxPool2d(2, 2) 

        # the 2nd conv. block 
        self.conv21 = nn.Conv2d(8, 16, 3,padding=1)
        
        self.conv22 = nn.Conv2d(16, 16, 3,padding=1)
        
        # the 3rd conv. block 
        self.conv31 = nn.Conv2d(16, 32, 3,padding=1)
        
        self.conv32 = nn.Conv2d(32, 32, 3,padding=1)
        
        # the 4th conv. block 
        self.conv41 = nn.Conv2d(56, 56, 3,padding=1)
        
        self.conv42 = nn.Conv2d(56, 2, 3,padding=1)
        
        # Upsampling
        self.ups24 = nn.Upsample(scale_factor=2)
        self.ups34 = nn.Upsample(scale_factor=4)
        if BN:
          self.bn11 = nn.BatchNorm2d(8)
          self.bn12 = nn.BatchNorm2d(8)
          self.bn21 = nn.BatchNorm2d(16)
          self.bn22 = nn.BatchNorm2d(16)
          self.bn31 = nn.BatchNorm2d(32)
          self.bn32 = nn.BatchNorm2d(32)
          self.bn41 = nn.BatchNorm2d(56)
          self.bn42 = nn.BatchNorm2d(2)
        if DP > 0:
          self.dropout2d = nn.Dropout2d(p=DP)
        
        self.fc1 = nn.Linear(32*30*17,200)
        self.fc2 = nn.Linear(200,1)
        # self.sm = nn.Sigmoid()

    def forward(self, x):
      # 1st block
      BN = self.BN
      DP = self.DP
      if x.size()[2]!=272 or x.size()[3]!=480:
        x = F.interpolate(x, size=(272,480))
      if BN:
        x1 = F.relu(self.bn11(self.conv11(x)))
        if DP > 0:
          x1 = self.dropout2d(x1)
        x1 = F.relu(self.bn12(self.conv12(x1)))
        if DP > 0:
          x1 = self.dropout2d(x1)
      else:
        x1 = F.relu(self.conv11(x))
        if DP > 0:
          x1 = self.dropout2d(x1)
        x1 = F.relu(self.conv12(x1))
        if DP > 0:
          x1 = self.dropout2d(x1)
      x1 = self.pool(x1)
      
      # 2nd Block
      if BN:
        x2 = F.relu(self.bn21(self.conv21(x1)))
        if DP > 0:
          x2 = self.dropout2d(x2)
        x2 = F.relu(self.bn22(self.conv22(x2)))
        if DP > 0:
          x2 = self.dropout2d(x2)
      else:
        x2 = F.relu(self.conv21(x1))
        if DP > 0:
          x2 = self.dropout2d(x2)
        x2 = F.relu(self.conv22(x2))
        if DP > 0:
          x2 = self.dropout2d(x2)
      
      # print(x2.size())
      x2 = self.pool(x2)
      
      # 3rd Block
      if BN:
        x3 = F.relu(self.bn31(self.conv31(x2)))
        if DP > 0:
          x3 = self.dropout2d(x3)
        x3 = F.relu(self.bn32(self.conv32(x3)))
        if DP > 0:
          x3 = self.dropout2d(x3)
      else:
        x3 = F.relu(self.conv31(x2))
        if DP > 0:
          x3 = self.dropout2d(x3)
        x3 = F.relu(self.conv32(x3))
        if DP > 0:
          x3 = self.dropout2d(x3)
      # print(x3.size())
      x3 = self.pool(x3)
      # print(x3.size())
      # 4th Block
      x2 = self.ups24(x2)
      x3u = self.ups34(x3)
      # print(x3up.size(),x2up.size(),x1.size())
      x4 = torch.cat([x2,x3u,x1],dim=1)
      if BN:
        x4 = F.relu(self.bn41(self.conv41(x4)))
        if DP > 0:
          x4 = self.dropout2d(x4)
        x4 = F.relu(self.bn42(self.conv42(x4)))
        if DP > 0:
          x4 = self.dropout2d(x4)
      else:
        x4 = F.relu(self.conv41(x4))
        if DP > 0:
          x4 = self.dropout2d(x4)
        x4 = F.relu(self.conv42(x4))
        if DP > 0:
          x4 = self.dropout2d(x4)
        
      out = F.log_softmax(x4,dim=1)
      x3 = x3.view(-1,32*30*17)
      x3 = self.fc2(F.relu(self.fc1(x3)))
      Cout = torch.sigmoid(x3)
      if out.size()[2]!=68 or out.size()[3]!=120:
        out = F.interpolate(out, size=(68,120))
      # print(out.size())
      del x1,x2,x3,x4
      torch.cuda.empty_cache()
      return out,Cout

Evaluation Function

In [0]:
def evaluate(model,data,modeltype):
  model.eval()
  Theta = []
  Acc = []
  theta = 0.94
  dtheta = 0.0004
  while theta < 1-0.0008 :
    
    theta = theta + dtheta
    acc = np.zeros(len(data), dtype=np.int16)
    # print(acc.sum())
    for i, (img, label) in enumerate(data):
      with torch.no_grad():
        Real_clas = np.max(label.cpu().numpy())>0
        real_pos = np.array(np.unravel_index(np.argmax(label.cpu().numpy(), axis=None), label.cpu().numpy().shape))
        img = (img.view(1,3,272,480)).to(device)
        if modeltype == 1:
          Mask, Cout= model(img.float()/255.0)
          Mask = torch.exp(Mask)
          a = Mask[0][0].cpu().numpy()
          pos = np.array(np.unravel_index(np.argmax(a, axis=None), a.shape))
          # p = a.max()*
          p = a.max()*Cout.cpu().numpy()
        else:
          Mask = model(img.float()/255.0)
          Mask = torch.exp(Mask)
          a = Mask[0][0].cpu().numpy()
          pos = np.array(np.unravel_index(np.argmax(a, axis=None), a.shape))
          p = a.max()
        # print(p, theta, pos,real_pos)
        if Real_clas:
          if (p>=theta) and np.max(np.abs(real_pos-pos)) < 5:
            acc[i] = 1
        else:
          if (p<theta):
            acc[i] = 1
    # print(theta, acc.mean())
    
    Theta.append(theta)
    Acc.append(acc.mean())
    print(theta*100)
  Acc = np.array(Acc)
  # print(Acc)
  # Theta = np.array(Thr)
  maxIdx = Acc.argmax()
  mTheta = Theta[maxIdx]
  mAcc = Acc.max()
  plt.figure(figsize=(15,10))
  plt.plot(Theta,Acc)
  plt.ylabel('Acc')
  plt.xlabel('theta')
  plt.grid(True)
  
  return mTheta, mAcc

Calculating accu.

In [0]:
DeepBallmodel = DeepBall().to(device)
DeepBallmodel.load_state_dict(torch.load(folderpath+'models/DeepBallmodel.pt')) #load the last checkpoint
mTheta, mAcc = evaluate(DeepBallmodel,test_dataset,2)
plt.title('DeepBall Model')
plt.show()
print(mTheta, mAcc)

C_DeepBallmodel = DeepBallClass(DP = 0.2).to(device)
C_DeepBallmodel.load_state_dict(torch.load(folderpath+'models/DeepBallclassmodel.pt')) #load the last checkpoint
CmTheta, CmAcc = evaluate(C_DeepBallmodel,test_dataset,1)
plt.title('C-DeepBall Model')
plt.show()
print(CmTheta, CmAcc)

CR_DeepBallmodel = DeepBallClass(DP = 0.2).to(device)
CR_DeepBallmodel.load_state_dict(torch.load(folderpath+'models/DeepBallclassmodel_r.pt')) #load the last checkpoint
CRmTheta, CRmAcc = evaluate(CR_DeepBallmodel,test_dataset,1)
plt.title('CR-DeepBall Model')
plt.show()
print(CRmTheta, CRmAcc)

R_DeepBallmodel = DeepBall().to(device)
R_DeepBallmodel.load_state_dict(torch.load(folderpath+'models/DeepBallmodel_r.pt')) #load the last checkpoint
RmTheta, RmAcc = evaluate(R_DeepBallmodel,test_dataset,2)
plt.title('R-DeepBall Model')
plt.show()
print(RmTheta, RmAcc)


Calcolate the acc with balls

In [0]:
def evaluate_pos(model,data,modeltype):
  model.eval()
  iteration = 0
  acc = 0
  # print(acc.sum())
  for i, (img, label) in enumerate(data):
    with torch.no_grad():
      Real_clas = np.max(label.cpu().numpy())>0
      real_pos = np.array(np.unravel_index(np.argmax(label.cpu().numpy(), axis=None), label.cpu().numpy().shape))
      img = (img.view(1,3,272,480)).to(device)
      if modeltype == 1:
        Mask, Cout= model(img.float()/255.0)
        Mask = torch.exp(Mask)
        a = Mask[0][0].cpu().numpy()
        pos = np.array(np.unravel_index(np.argmax(a, axis=None), a.shape))
        # p = a.max()*
        p = a.max()*Cout.cpu().numpy()
      else:
        Mask = model(img.float()/255.0)
        Mask = torch.exp(Mask)
        a = Mask[0][0].cpu().numpy()
        pos = np.array(np.unravel_index(np.argmax(a, axis=None), a.shape))
        p = a.max()
      # print(p, theta, pos,real_pos)
      
      if Real_clas:
        iteration += 1
        if np.max(np.abs(real_pos-pos)) < 5:
          acc += 1
  print('>> Acc: {:.3f}, with_ball  {}/{} '.format(acc/iteration*100, iteration,len(data)))  
  return acc/iteration

In [0]:
DeepBallmodel = DeepBall().to(device)
DeepBallmodel.load_state_dict(torch.load(folderpath+'models/DeepBallmodel.pt')) #load the last checkpoint
TruAcc = evaluate_pos(DeepBallmodel,test_dataset,2)
# print(TruAcc)

C_DeepBallmodel = DeepBallClass(DP = 0.2).to(device)
C_DeepBallmodel.load_state_dict(torch.load(folderpath+'models/DeepBallclassmodel.pt')) #load the last checkpoint
CTruAcc = evaluate_pos(C_DeepBallmodel,test_dataset,1)
# print(CTruAcc)

R_DeepBallmodel = DeepBall().to(device)
R_DeepBallmodel.load_state_dict(torch.load(folderpath+'models/DeepBallmodel_r.pt')) #load the last checkpoint
RTruAcc = evaluate_pos(R_DeepBallmodel,test_dataset,2)
# print(RTruAcc)

CR_DeepBallmodel = DeepBallClass(DP = 0.2).to(device)
CR_DeepBallmodel.load_state_dict(torch.load(folderpath+'models/DeepBallclassmodel_r.pt')) #load the last checkpoint
CRTruAcc = evaluate_pos(CR_DeepBallmodel,test_dataset,1)
# print(CRTruAcc)

>> Acc: 72.408, with_ball  569/5990 
>> Acc: 81.547, with_ball  569/5990 
>> Acc: 75.747, with_ball  569/5990 
>> Acc: 80.844, with_ball  569/5990 


Results in Table

In [0]:
T = tabulate([['DeepBall',mTheta, mAcc, TruAcc],
              ['C-DeepBall',CmTheta, CmAcc, CTruAcc], 
              ['R-DeepBall',RmTheta, RmAcc, RTruAcc],
              ['CR-DeepBall',CRmTheta, CRmAcc, CRTruAcc]]
             ,headers=['Model', 'Threshold', 'Accuracy'. 'pos Accuracy'])
print(T)